In [1]:
from enum import Enum
import numpy as np
import pandas as pd
import math as m
from pathlib import Path

base_directory = "D:/Descargas/i2ascii-files/i2ascii-files/"

class STATE(Enum):
    EVENT_ID = 1
    RUN_INFO = 2
    WEIGHTS = 3
    NEUTRINO = 4
    MUON = 5
    AAFIT = 6
    BBFIT_TRACK = 7
    BBFIT_BRIGHT = 8
    GRIDFIT = 9
    HITS = 10
    SELECTED_HITS = 11
    
def event_id (line, event, state, df_list):
    if "start_event" in line:
        event["event_id"] = line.split()[1]
        state = STATE.RUN_INFO
      #  print("START EVENT: " + event["event_id"])
    return event, state
        
def run_info (line, event, state, df_list):
    element_list = line.split()
    if len(element_list) == 7: 
        event["run_id"] = element_list[0]
        event["frame_id"] = element_list[1]
        event["trigger_counter"] = element_list[2]
        event["date"] = element_list[4]
        event["time"] = element_list[5].split(",")[0]
        state = STATE.WEIGHTS
    else:
        print("ERROR IN STATE: RUN_INFO")
    return event, state

def weights (line, event, state, df_list):
    element_list = line.split()
    if len(element_list) == 4 and element_list[0] == "weights": 
        event["w_1"] = element_list[1]
        event["w_2"] = element_list[2]
        event["w_3"] = element_list[3]
        state = STATE.NEUTRINO
    else:
        print("ERROR IN STATE: WEIGHTS")
    return event, state

def cart2sph(x,y,z):
    x = float(x)
    y = float(y)
    z = float(z)
    XsqPlusYsq = x**2 + y**2
    r = m.sqrt(XsqPlusYsq + z**2)               # r
    elev = m.atan2(z,m.sqrt(XsqPlusYsq))     # theta
    az = m.atan2(y,x)                           # phi
    return elev, az

def neutrino (line, event, state, df_list):
    element_list = line.split()
    if len(element_list) == 9 and element_list[0] == "nu": 
    #    event["n_elev"], event["n_az"] = cart2sph(element_list[1], element_list[2], element_list[3])
        # event["n_dir_x"] = element_list[1]
        # event["n_dir_y"] = element_list[2]
        # event["n_dir_z"] = element_list[3]
    #    event["n_pos_x"] = element_list[4]
    #    event["n_pos_y"] = element_list[5]
    #    event["n_pos_z"] = element_list[6]
    #    event["n_energy"] = element_list[7]
        state = STATE.MUON
    else:
        print("ERROR IN STATE: NEUTRINO")
    return event, state
   
def muon (line, event, state, df_list):
    element_list = line.split()
    if len(element_list) == 9 and element_list[0] == "muon": 
        event["m_elev"], event["m_az"] = cart2sph(element_list[1], element_list[2], element_list[3])
        #event["m_dir_x"] = element_list[1]
        #event["m_dir_y"] = element_list[2]
        #event["m_dir_z"] = element_list[3]
        #event["m_pos_x"] = element_list[4]
        #event["m_pos_y"] = element_list[5]
        #event["m_pos_z"] = element_list[6]
        #event["m_energy"] = element_list[7]
        state = STATE.AAFIT
    else:
        print("ERROR IN STATE: MUON")
    return event, state

def aafit (line, event, state, df_list):
    element_list = line.split()
    if len(element_list) == 9 and element_list[0] == "aafit":
        event["aafit_elev"], event["aafit_az"] = cart2sph(element_list[1], element_list[2], element_list[3])
        #event["aafit_dir_x"] = element_list[1]
        #event["aafit_dir_y"] = element_list[2]
        #event["aafit_dir_z"] = element_list[3]
        event["aafit_lambda"] = element_list[7]
        event["aafit_beta"] = element_list[8]
        state = STATE.BBFIT_TRACK
    else:
        state = STATE.EVENT_ID
        print("ERROR IN STATE: AAFIT -> Event " + event["event_id"])
    return event, state
    
def bbfit_track (line, event, state, df_list):
    element_list = line.split()
    if len(element_list) == 8 and element_list[0] == "bbfit_track":
        event["bbfit_elev"], event["bbfit_az"] = cart2sph(
            0 if element_list[1] == "nan" else element_list[1],
            0 if element_list[2] == "nan" else element_list[2],
            0 if element_list[3] == "nan" else element_list[3])
        # event["bbfit_dir_x"] = np.nan if element_list[1] == "nan" else element_list[1]
        event["bbfit_dir_x"] = 0 if element_list[1] == "nan" else element_list[1]
        event["bbfit_dir_y"] = 0 if element_list[2] == "nan" else element_list[2]
        event["bbfit_dir_z"] = 0 if element_list[3] == "nan" else element_list[3]
        event["bbfit_chi2"] = element_list[7]
        state = STATE.BBFIT_BRIGHT
    else:
        print("ERROR IN STATE: BBFIT_TRACK")
    return event, state

def bbfit_bright (line, event, state, df_list):
    element_list = line.split()
    if len(element_list) == 8 and element_list[0] == "bbfit_bright": 
        state = STATE.GRIDFIT
    else:
        print("ERROR IN STATE: BBFIT_BRIGHT")
    return event, state
    
def gridfit (line, event, state, df_list):
    element_list = line.split()
    if len(element_list) == 8 and element_list[0] == "gridfit": 
        state = STATE.HITS
    elif element_list[0] == "hit":
        # Some times there are not gridfit measures...
        return hits(line, event, state, df_list)
    else:
        print("ERROR IN STATE: GRIDFIT")
    return event, state

def hits (line, event, state, df_list):
    element_list = line.split()
    if len(element_list) == 14 and element_list[0] == "hit":
        state = STATE.HITS
    elif len(element_list) == 3 and element_list[0] == "BBFit": 
        state = STATE.SELECTED_HITS
    else:
        print("ERROR IN STATE: HITS")
    return event, state
   
def selected_hits (line, event, state, df_list):
    element_list = line.split()
    if len(element_list) == 14 and element_list[0] == "hit": 
        state = STATE.SELECTED_HITS
    elif "end_event" in line:
        state = STATE.EVENT_ID
        temp_event = event.copy()
        df_list.append(temp_event)
        event.clear()
        event['det_type'] = temp_event['det_type']
        #print("END EVENT: " + event["event_id"])
    else:
        print("ERROR IN STATE: SELECTED_HITS")
    return event, state
   
state_dict = {
    STATE.EVENT_ID: event_id,
    STATE.RUN_INFO: run_info,
    STATE.WEIGHTS: weights,
    STATE.NEUTRINO: neutrino,
    STATE.MUON: muon,
    STATE.AAFIT: aafit,
    STATE.BBFIT_TRACK: bbfit_track,
    STATE.BBFIT_BRIGHT: bbfit_bright,
    STATE.GRIDFIT: gridfit,
    STATE.HITS: hits,
    STATE.SELECTED_HITS: selected_hits
}

In [2]:
pathlist = Path(base_directory).glob('*.txt')
counter = 0;
df_list = []
for path in pathlist:
    # because path is object not string
    path_in_str = str(path)
    print("\nFILE: " + path_in_str)
    detection_type = 1 if "anumu" in path_in_str else 2
    with open(path_in_str, "r") as f_test:
        line = f_test.readline()
        current_state = STATE.EVENT_ID
        current_event = {}
        current_event['det_type'] = detection_type
        while line:
            current_event, current_state = state_dict[current_state](line, current_event, current_state, df_list)
            line = f_test.readline()
            
    counter += 1
    if counter > 1:
        break;
    
df = pd.DataFrame(df_list)
print(df.shape)


FILE: D:\Descargas\i2ascii-files\i2ascii-files\MC_025800_anumu_CC_a_reco.i3.gz.txt

FILE: D:\Descargas\i2ascii-files\i2ascii-files\MC_025800_numu_CC_a_reco.i3.gz.txt
     aafit_az        aafit_beta  aafit_elev    aafit_lambda  bbfit_az  \
0   -2.345721    0.130539349905    1.136707  -5.86985595954  0.000000   
1    1.572092   0.0836670029347    1.037053   -5.7793266451  0.000000   
2    0.491548   0.0202099344442    0.387720  -5.75199073111  0.000000   
3    1.300769  0.00798271375863    1.384257  -6.10337920021  0.000000   
4    2.791342  0.00972042559206    0.827787  -5.10942573606  2.794801   
5    1.456237   0.0209666765195    0.776675  -5.54507124415  1.582917   
6    2.814358  0.00818518948302    0.839293   -6.3655854932  0.000000   
7   -0.894596   0.0106758698475    0.991760  -5.30778362015  0.000000   
8    2.260261    0.011746644012    0.896976  -5.52212398141  2.225389   
9   -0.782032   0.0415682655896    0.516033  -5.40763571919  0.000000   
10  -2.183115    0.11696860481

In [3]:
print(df.shape)

(479, 22)


In [56]:
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import sklearn.metrics as metrics
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict

# Métricas de evaluación que se utilizarán. No nos preocupamos en negarlas debido a que no se utilizarán de forma automática,
# solo las visualizaremos
metrics_dict = {
  'MAE': metrics.mean_absolute_error,
  'RMSE': lambda y, y_pred:
          sqrt(metrics.mean_squared_error(y, y_pred)),
  'MAPE': lambda y, y_pred:
          np.mean(np.abs((y - y_pred) / y)) * 100,
  'E_VARIANCE': metrics.explained_variance_score
}

algorithms_dict = {}

# Algoritmo 1: Estandarización + OLS
steps = [('standardization', StandardScaler()),
         ('ols', LinearRegression())]
algorithms_dict['STD_OLS'] = Pipeline(steps)

# Algoritmo 2: Estandarización + KNN
steps = [('standardization', StandardScaler()),
         ('knn', KNeighborsRegressor(n_neighbors=10))]
algorithms_dict['STD_KNN'] = Pipeline(steps)

# Algoritmo 3: Estandarización + RandomForest
steps = [('standardization', StandardScaler()),
         ('rforest', RandomForestRegressor())]
algorithms_dict['STD_RF'] = Pipeline(steps)

In [57]:
target = df[['m_elev','m_az']]
data2 = df.drop(['m_elev', 'm_az'], axis=1)
data = data2.drop(['event_id', 'run_id', 'date', 'time'], axis=1)

In [41]:
data.tail

<bound method NDFrame.tail of        aafit_az        aafit_beta  aafit_elev    aafit_lambda  bbfit_az  \
0     -2.345721    0.130539349905    1.136707  -5.86985595954  0.000000   
1      1.572092   0.0836670029347    1.037053   -5.7793266451  0.000000   
2      0.491548   0.0202099344442    0.387720  -5.75199073111  0.000000   
3      1.300769  0.00798271375863    1.384257  -6.10337920021  0.000000   
4      2.791342  0.00972042559206    0.827787  -5.10942573606  2.794801   
5      1.456237   0.0209666765195    0.776675  -5.54507124415  1.582917   
6      2.814358  0.00818518948302    0.839293   -6.3655854932  0.000000   
7     -0.894596   0.0106758698475    0.991760  -5.30778362015  0.000000   
8      2.260261    0.011746644012    0.896976  -5.52212398141  2.225389   
9     -0.782032   0.0415682655896    0.516033  -5.40763571919  0.000000   
10    -2.183115    0.116968604817    1.082496  -4.95486163107  0.000000   
11     1.815705    0.024691221056    1.103998  -5.37431636339  0.00000

In [58]:
# Se realizan las 6 predicciones y se almacenan en un diccionario con nombres descriptivos 
y_pred_dict = {}
for alg_name, alg in algorithms_dict_3.items():
    y_pred_dict[alg_name] = cross_val_predict(alg, data, target, 
                                                              cv=KFold(n_splits=10, random_state=1))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with i

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Da

KeyboardInterrupt: 

In [ ]:
# En lugar de importar la función del script utilizado en clase, se reimplementa con ligeras variaciones para
# adaptarse a las necesidades del ejercicio
def grafica_real_vs_pred_mod(y_true, y_pred, evaluated_metrics, algorithm_name):
    fig, ax = plt.subplots()
    ax.scatter(y_true, y_pred, edgecolors=(0, 0, 0))
    ax.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'k--', lw=4)
    ax.set_xlabel('Real class value')
    ax.set_ylabel('Prediction')
    title = algorithm_name + '\n'
    for name, result in evaluated_metrics.items():
        title += name + ': ' + str(round(result, 3)) + ' '
        
    plt.title(title)
    plt.show()

In [ ]:
visual_comp = target.join(pd.DataFrame(y_pred_dict['STD_RF']))


In [ ]:
visual_comp

In [ ]:
# TEST 1
# Se evaluan las predicciones y se representan al estilo visto en clase
evaluated = {}
for name, prediction in y_pred_dict.items():
    for metric_name, metric in metrics_dict.items():
        evaluated[metric_name] = metric(target, prediction)

    grafica_real_vs_pred_mod(target, prediction, evaluated, name)